## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,city,country,date,lat,lng,max_temp,humidity,cloudiness,wind speed
0,0,Barrow,US,2021-01-12 00:09:38,71.2906,-156.7887,1.40,78,90,23.02
1,1,Chiredzi,ZW,2021-01-12 00:15:36,-21.0500,31.6667,73.09,92,70,6.73
2,2,Khatanga,RU,2021-01-12 00:04:27,71.9667,102.5000,-19.35,88,100,4.85
3,3,Taltal,CL,2021-01-12 00:15:37,-25.4000,-70.4833,65.73,73,53,5.37
4,4,Hermanus,ZA,2021-01-12 00:02:49,-34.4187,19.2345,66.99,92,90,3.00


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the min temperature you would prefer for your travels?"))
max_temp = float(input("What is the max temperature you would prefer for your travels?"))

What is the min temperature you would prefer for your travels?75
What is the max temperature you would prefer for your travels?90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
ideal_cities_df = city_data_df.loc[(city_data_df["max_temp"]<=max_temp)&(city_data_df["max_temp"]>=min_temp)]
ideal_cities_df.head()

,City_ID,city,country,date,lat,lng,max_temp,humidity,cloudiness,wind speed
6,6,Georgetown,MY,2021-01-12 00:04:33,5.4112,100.3354,78.80,78,20,3.44
8,8,Rikitea,PF,2021-01-12 00:09:37,-23.1203,-134.9692,78.64,76,92,19.08
13,13,Butaritari,KI,2021-01-12 00:09:37,3.0707,172.7902,81.14,82,83,19.55
17,17,Faanui,PF,2021-01-12 00:05:14,-16.4833,-151.7500,81.81,75,91,21.00
23,23,Axim,GH,2021-01-12 00:15:41,4.8699,-2.2405,79.09,87,0,7.85


In [10]:
# 4a. Determine if there are any empty rows.
ideal_cities_df.count()

City_ID       185
city          185
country       185
date          185
lat           185
lng           185
max_temp      185
humidity      185
cloudiness    185
wind speed    185
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = ideal_cities_df

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["city", "country", "max_temp","lat", "lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,city,country,max_temp,lat,lng,Hotel Name
6,Georgetown,MY,78.80,5.4112,100.3354,
8,Rikitea,PF,78.64,-23.1203,-134.9692,
13,Butaritari,KI,81.14,3.0707,172.7902,
17,Faanui,PF,81.81,-16.4833,-151.7500,
23,Axim,GH,79.09,4.8699,-2.2405,
24,Puerto Ayora,EC,75.00,-0.7393,-90.3518,
26,Freetown,SL,80.08,8.4840,-13.2299,
27,Kavieng,PG,84.79,-2.5744,150.7967,
28,Kapaa,US,78.80,22.0752,-159.3190,
36,Saint-Philippe,RE,77.00,-21.3585,55.7679,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    latitude = row["lat"]
    longitude = row["lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{latitude},{longitude}"
    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url,params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df
clean_hotel_df

,city,country,max_temp,lat,lng,Hotel Name
6,Georgetown,MY,78.80,5.4112,100.3354,Cititel Penang
8,Rikitea,PF,78.64,-23.1203,-134.9692,Pension Maro'i
13,Butaritari,KI,81.14,3.0707,172.7902,Isles Sunset Lodge
17,Faanui,PF,81.81,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
23,Axim,GH,79.09,4.8699,-2.2405,AXIM BEACH RESORT AND SPA
...,...,...,...,...,...,...
653,La Asuncion,VE,76.77,11.0333,-63.8628,Colinas de Matasiete
658,Le Port,RE,78.80,-20.9373,55.2919,Villa Maido
666,Eirunepe,BR,76.15,-6.6603,-69.8736,Atrás de você
667,Cooma,AU,80.60,-36.2333,149.1333,High Country Motel


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
<dt>Max Temp</dt><dd>{max_temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
hotel_info
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["lat", "lng"]]
locations
max_temp = clean_hotel_df["max_temp"]
fig = gmaps.figure(center=(30.0,31.0),zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300,point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# 11a. Add a marker layer for each city to the map. 
